# [Kaggle Digit Recognizer Challenge](https://www.kaggle.com/competitions/digit-recognizer)
From https://www.datacamp.com/tutorial/xgboost-in-python

In [4]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import xgboost as xgb

mnist_train = pd.read_csv("/home/rainer/Downloads/ML_datasets/mnist_train.csv")

mnist_test = pd.read_csv("/home/rainer/Downloads/ML_datasets/mnist_test.csv")

X, Y = mnist_train.drop('label', axis=1), mnist_train[['label']]

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=1)
dtrain_reg = xgb.DMatrix(X_train, Y_train, enable_categorical=False)
dtest_reg = xgb.DMatrix(X_test, Y_test, enable_categorical=False)
dfull = xgb.DMatrix(X, Y, enable_categorical=False)

In [6]:
# Define hyperparameters
params = {
    "objective": "multi:softmax",
    "tree_method": "gpu_hist",
    "num_class": Y['label'].nunique()
}
n=100
results = xgb.cv(
    params=params,
    dtrain=dtrain_reg,
    num_boost_round=n,
    nfold=5,
    verbose_eval=10, # print eval every xth round
    metrics=["mlogloss", "merror"],
)

[0]	train-logloss:-127.63251+0.33649	train-error:-3.46438+0.00913	test-logloss:-127.63251+1.34597	test-error:-3.46438+0.03653


XGBoostError: [21:56:45] ../src/tree/updater_gpu_hist.cu:802: Exception in gpu_hist: [21:56:45] ../src/c_api/../data/../common/device_helpers.cuh:431: Memory allocation error on worker 0: std::bad_alloc: cudaErrorMemoryAllocation: out of memory
- Free memory: 197394432
- Requested memory: 268435456

Stack trace:
  [bt] (0) /home/rainer/miniconda3/envs/jupyter_standard/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x451c73) [0x7f8671851c73]
  [bt] (1) /home/rainer/miniconda3/envs/jupyter_standard/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x456f71) [0x7f8671856f71]
  [bt] (2) /home/rainer/miniconda3/envs/jupyter_standard/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x11256f) [0x7f867151256f]
  [bt] (3) /home/rainer/miniconda3/envs/jupyter_standard/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x70594d) [0x7f8671b0594d]
  [bt] (4) /home/rainer/miniconda3/envs/jupyter_standard/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x712b30) [0x

In [ ]:
model = xgb.train(
    params=params,
    dtrain=dfull,
    num_boost_round=100
)
final_test = xgb.DMatrix(mnist_test, enable_categorical=False)
predictions = model.predict(final_test)

In [ ]:
predictionsDf = pd.DataFrame(predictions, columns=["Label"])
predictionsDf.insert(0, 'ImageId', range(1, len(predictionsDf) + 1))
predictionsDf["Label"] = predictionsDf["Label"].astype(int)

In [ ]:
predictionsDf

In [ ]:
predictionsDf.to_csv('/home/rainer/Downloads/ML_datasets/mnist_test_prediction.csv', index=False)